In [2]:
import os
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from sympy.codegen.cnodes import static
from torch.utils.data import Dataset, DataLoader

In [ ]:
# data paths
mixture_directory = "data/mixture"
vocals_directory = "data/vocals"

In [ ]:
# parameters to play around with to better optimize training
patch_size = 128
stride = 64
batch_size = 4
epochs = 20
model_save_path = "vocal_isolator.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # will utilize GPU if possible to train/test

In [1]:
class SpectrogramDataset(Dataset):
    def __init__(self, mixed_dir, vocal_dir, use_magnitude=True):
        self.mixed_map = self._make_file_map(mixed_dir, prefix='mix_')
        self.vocals_map = self._make_file_map(vocal_dir, prefix='vocals_')
        
        # error check with available song keys
        self.keys = sorted(list(set(self.mixed_map.keys()) & set(self.vocals_map.keys())))
        assert self.keys, "No matching files found between mixed and vocal directories"
        
        self.pairs = [(self.mixed_map[k], self.vocals_map[k]) for k in self.keys]
        self.use_magnitude = use_magnitude
    
    @staticmethod
    def _make_file_map(directory, prefix):
        file_map = {}
        for path in glob.glob(os.path.join(directory, "*.csv")):
            filename = os.path.basename(path)
            if filename.startswith(prefix):
                key = filename[len(prefix):]  # strip the prefix
                file_map[key] = path
        return file_map
    
    @staticmethod
    def _load_excel_sheet(filepath):
        real = pd.read_excel(filepath, sheet_name='Re_X', header=None).values.astype(np.float32)
        imaginary = pd.read_excel(filepath, sheet_name='Im_X', header=None).values.astype(np.float32)
        fs = pd.read_excel(filepath, sheet_name='Sampling Rate', header=None).values.[0][0]
        
        complex_stft = real + 1j*imaginary
        return complex_stft, fs
    
    def __len__(self):
        return len(self.pairs) #length of le dataset?BRO THIS IS WH Y IM NOT IN COMPUTER ENGINE
        
    def __getitem__(self, index):
        mixed_path, vocals_path = self.pairs[index]

        mixed_stft, _ = self._load_excel_sheet(mixed_path)
        vocals_stft, _ = self._load_excel_sheet(vocals_path)
        
        if self.use_magnitude:
            mixed_mag = np.abs(mixed_stft)
            vocals_mag = np.abs(vocals_stft)
            
            mixed_mag = (mixed_mag - mixed_mag.min()) / (mixed_mag.max() - mixed_mag.min() + 1e-8)
            vocals_mag = (vocals_mag - vocals_mag.min()) / (vocals_mag.max() - vocals_mag.min() + 1e-8)
            
            mixed_tensor = torch.tensor(mixed_mag).unsqueeze(0)
            vocals_tensor = torch.tensor(vocals_mag).unsqueeze(0)
            
            return mixed_tensor, vocals_tensor

        else:  # if training on complex values
            return torch.tensor(mixed_stft), torch.tensor(vocals_stft)


IndentationError: expected an indented block after function definition on line 22 (2098168021.py, line 24)

In [ ]:
class UNet(nn.Module):  #TODO: Should we be using a CNN or a UNET?
    def __init__(self):
        super(UNet, self).__init__()
        
        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),  # 3x3 filter for feature detection
                nn.BatchNorm2d(out_channels),  # stabilize and speeds up training
                nn.ReLU(inplace=True),  # apply non-linearity for complex pattern learning
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
            )
        
        # Encoding - reduce spatial dimensions and abstract features so model can understand
        self.encoder1 = conv_block(1, 64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # reduce resolution by 2 to allow for larger context
        self.encoder2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = conv_block(256, 512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.center = conv_block(512, 1024)  # decision hub - learns what high level features are
        
        #Decoding - up sample and reconstruct the isolated vocals
        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = conv_block(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = conv_block(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = conv_block(128, 64)
            
        self.final = nn.Conv2d(64, 1, kernel_size=1)  # reduce back to 1 channel (vocal spectrogram)
        
    def forward(self, x):
        # encoding
        e1 = self.encoder1(x)
        e2 = self.encoder2(self.pool1(e1))
        e3 = self.encoder3(self.pool2(e2))
        e4 = self.encoder4(self.pool3(e3))
        center = self.center(self.pool4(e4))
        
        # decoding
        d4 = self.dec4(torch.cat([self.up4(center), e4], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d4), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))
        out = self.final(d1)
        
        return out

In [ ]:
# training function to be called
def train(model, dataloader, criterion, optimizer, num_epochs, checkpoint_path="checkpoints"):
    os.makedirs(checkpoint_path, exist_ok=True)
    model.train()
    checkpoint_interval = 5

    for epoch in range(1, num_epochs + 1):
        running_loss = 0.0
        num_batches = 0

        for mixed, vocals in dataloader:
            # Move to GPU
            mixed = mixed.to(device)
            vocals = vocals.to(device)

            # Forward pass
            outputs = model(mixed)
            loss = criterion(outputs, vocals)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            num_batches += 1

        # Calculate average loss
        epoch_loss = running_loss / num_batches

        print(f"Epoch {epoch}/{num_epochs} — Average Loss: {epoch_loss:.4f}")

        # Periodic checkpoints
        if epoch % checkpoint_interval == 0:
            ckpt = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
            }
            torch.save(ckpt, os.path.join(checkpoint_path, f"sep_epoch_{epoch}.pth"))
            
        torch.save(model.state_dict(), os.path.join(checkpoint_path, f"final_model.pth"))

    return model

In [ ]:
# training block
dataset = SpectrogramDataset(mixed_dir=mixture_directory, vocal_dir=vocals_directory, use_magnitude=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = UNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # load model onto device

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train(model=model, dataloader=dataloader, criterion=criterion, optimizer=optimizer, num_epochs=epochs, checkpoint_path=model_save_path)